In [1]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.

device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [2]:
!pip install torch
!pip install transformers
!pip install -U -q PyDrive
!pip install datasets
!pip install seqeval
!pip install ray[tune]
!pip install transformers accelerate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=15) # num_labels = 14 + 1


Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

# Importing Datasets

In [4]:
import pandas as pd
import json
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

train_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_JUDGEMENT_PREPROCESSED.json')
train_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_PREAMBLE_PREPROCESSED.json')

test_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_JUDGEMENT_PREPROCESSED.json')
test_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_PREAMBLE_PREPROCESSED.json')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [5]:
print(len(train_df_judgement), len(train_df_preamble))
print(len(test_df_judgement), len(test_df_preamble))
total_data = len(train_df_judgement) + len(train_df_preamble) + len(test_df_judgement) + len(test_df_preamble)

8494 553
839 28


# Train : Validation : Test split (70:20:10)

In [6]:
# concatenating the judgement + preamble data
df = pd.concat([train_df_judgement, train_df_preamble])
df.reset_index(inplace=True, drop=True)
# print(df)
print(total_data)
# splitting train (70% of the total data)
split=int(total_data*0.7)
print(split)
train_df = df[:split]
# splitting validation (appx 20% of the total data)
val_df = df[split:]
# test data (appx 10% of the total data)
test_df = pd.concat([test_df_judgement, test_df_preamble])

9914
6939


In [7]:
len(train_df)

6939

In [8]:
len(val_df)

2108

In [9]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization

In [10]:
labels_list = [" OTHERS", " PETITIONER", " COURT", " RESPONDENT", " JUDGE", " OTHER_PERSON", " LAWYER", " DATE", " ORG", " GPE", " STATUTE", " PROVISION", " PRECEDENT", " CASE_NUMBER", " WITNESS"]
label_encoding_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}
label_list_encoding_dict = {0: " OTHERS", 1: " PETITIONER", 2: " COURT", 3: " RESPONDENT", 4: " JUDGE", 5: " OTHER_PERSON", 6: " LAWYER", 7: " DATE", 8: " ORG", 9: " GPE", 10: " STATUTE", 11: " PROVISION", 12: " PRECEDENT", 13: " CASE_NUMBER", 14: " WITNESS"}

In [11]:
def tokenize_all_labels(rows):
    tokenized_inputs = tokenizer(list(rows["tokens"]), truncation = True, is_split_into_words = True)
    labels, label_all = [], True
    for index, label in enumerate(rows["ner_tags"]):
        # print(i, label)
        prior_idx = None
        word_ids = tokenized_inputs.word_ids(batch_index = index)
        
        label_ids = []
        for current_idx in word_ids:
            if current_idx is None: label_ids.append(-100)
            elif label[current_idx] == '0': label_ids.append(0)
            elif current_idx != prior_idx: label_ids.append(label_encoding_dict[label[current_idx]])
            else: label_ids.append(label_encoding_dict[label[current_idx]] if label_all else -100)
            prior_idx = current_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
train_dataset_tokenized = train_dataset.map(tokenize_all_labels, batched=True)
val_dataset_tokenized = val_dataset.map(tokenize_all_labels, batched=True)

Map:   0%|          | 0/6939 [00:00<?, ? examples/s]

Map:   0%|          | 0/2108 [00:00<?, ? examples/s]

# Training

In [13]:
import wandb
wandb.login()

wandb: Currently logged in as: astha (685_data_augmentation_legal_ner). Use `wandb login --relogin` to force relogin


True

In [14]:
import warnings
warnings.filterwarnings('ignore')

# hyperparameters
batch_size = 5
epoch = 5
learning_rate = 2.910635913133073e-05

# tokenizing and loading legalBERT model 
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=len(labels_list))
metric = load_metric("seqeval")

training_arguments = TrainingArguments(
    "eval_indian_legal_ner",
    report_to='wandb',
    evaluation_strategy = "steps",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=1e-5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    run_name='gold'  
)

# training on train set and evaluating on validation set
data_collator = DataCollatorForTokenClassification(tokenizer)

best_accuracy = 0
best_results = 0

def evaluate_metrics(pred_tuple):
    global best_accuracy
    global best_results
    predictions, labels = pred_tuple
    predictions = np.argmax(predictions, axis=2)
    
    actual_predictions = [[labels_list[pred_tuple] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    actual_labels = [[labels_list[l] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=actual_predictions, references=actual_labels)
    if best_accuracy < results["overall_accuracy"]: 
      best_results = results
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    training_arguments,
    train_dataset = train_dataset_tokenized,
    eval_dataset = val_dataset_tokenized,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=evaluate_metrics
)
trainer.train()
trainer.evaluate()
trainer.save_model('indian_legal_ner.model')
wandb.finish()

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.330000,0.700339,0.408479,0.597361,0.485185,0.767767
1000,0.167600,0.447778,0.463045,0.649170,0.540534,0.820037
1500,0.130300,0.701066,0.461899,0.649850,0.539987,0.768860
2000,0.093400,0.607251,0.458172,0.655186,0.539248,0.771642
2500,0.094500,0.558767,0.467162,0.670806,0.550763,0.800535
3000,0.077700,0.674610,0.497071,0.683225,0.575468,0.786060
3500,0.062200,0.545491,0.512112,0.703502,0.592741,0.822099
4000,0.053600,0.763344,0.503907,0.682061,0.579603,0.782002
4500,0.037000,0.746634,0.491416,0.677598,0.569681,0.781422
5000,0.032400,0.754353,0.504484,0.671291,0.576055,0.783054


eval/accuracy,▁█▁▁▅▃█▃▃▃▇▇▆█
eval/f1,▁▅▄▄▅▇█▇▆▇████
eval/loss,▇▁▇▅▃▆▃███▅▆▆▃
eval/precision,▁▅▄▄▅▇█▇▆▇████
eval/recall,▁▄▄▅▆▆█▆▆▆████
eval/runtime,▂▁▁▁▄▆▂▁▆▁█▁▁▇
eval/samples_per_second,▇█▇█▅▃▇█▃█▁██▂
eval/steps_per_second,▇█▇█▅▃▇█▃█▁██▂
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇████
train/learning_rate,█▇▇▆▆▅▅▄▃▃▂▂▁


In [15]:
best_results

{'CASE_NUMBER': {'precision': 0.17073170731707318,
  'recall': 0.8296296296296296,
  'f1': 0.2831858407079646,
  'number': 135},
 'COURT': {'precision': 0.4656964656964657,
  'recall': 0.6511627906976745,
  'f1': 0.5430303030303031,
  'number': 344},
 'DATE': {'precision': 0.4683760683760684,
  'recall': 0.9547038327526133,
  'f1': 0.6284403669724771,
  'number': 287},
 'GPE': {'precision': 0.24319419237749546,
  'recall': 0.7243243243243244,
  'f1': 0.3641304347826087,
  'number': 185},
 'JUDGE': {'precision': 0.7231404958677686,
  'recall': 0.8254716981132075,
  'f1': 0.7709251101321587,
  'number': 212},
 'LAWYER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 70},
 'ORG': {'precision': 0.33755274261603374,
  'recall': 0.7174887892376681,
  'f1': 0.45911047345767575,
  'number': 223},
 'OTHERS': {'precision': 0.6019353995030731,
  'recall': 0.752370055573717,
  'f1': 0.668797675263349,
  'number': 6118},
 'OTHER_PERSON': {'precision': 0.28293736501079914,
  'recall': 0.8086

# Evaluating Test Dataset

In [16]:
wandb.init(name = "gold-test")

In [17]:
test_dataset_tokenized = test_dataset.map(tokenize_all_labels, batched=True)
test_results = trainer.evaluate(test_dataset_tokenized)
test_results

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

{'eval_loss': 0.21933990716934204,
 'eval_precision': 0.7557046979865771,
 'eval_recall': 0.8440779610194903,
 'eval_f1': 0.7974504249291786,
 'eval_accuracy': 0.9219717612088184,
 'eval_runtime': 6.4736,
 'eval_samples_per_second': 133.928,
 'eval_steps_per_second': 26.878,
 'epoch': 5.0}

In [18]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.92197


# Evaluating legalBART Samples

In [19]:
wandb.init(name = "legalbart-test")

In [20]:
# loading legalBART samples
legalBARTSamples = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/legal_BART_Samples.csv')
legalBARTSamples['tokens'] = legalBARTSamples['tokens'].map(lambda x: eval(x))
legalBARTSamples['ner_tags'] = legalBARTSamples['ner_tags'].map(lambda x: eval(x))

In [21]:
legalBARTSamples

,tokens,ner_tags
0,"[On, specific, query, by, the, Bench, ,, it, w...","[0, 0, 0, 0, 5, 1, 5, 7, 6, 0, 0, 0, 0, 5, 0, ..."
1,"[According, to, the, Agya, ,, span, class, '',...","[0, 5, 5, 3, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, ..."
2,"[PW3, Vijay, Mishra, ,, Deputy, Manager, ,, HD...","[0, 7, 7, 5, 2, 0, 5, 0, 2, 5, 5, 5, 0, 0, 0, ..."
3,"[He, was, asked, to, come, and, carry, out, th...","[0, 6, 0, 5, 0, 5, 0, 0, 5, 1, 0, 0, 0, 0, 6, ..."
4,"[The, pillion, rider, ,, Satyanarayana, Murthy...","[5, 0, 0, 5, 3, 3, 6, 0, 0, 5, 5, 0, 5, 5, 0, ..."
...,...,...
8389,"[The, Honble, Sri, Justice, Nooty, Ramamohana,...","[2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 1, ..."
8390,"[Petitioner, :, Indira, Nehru, Gandhi, Respond...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, ..."
8391,"[In, The, Court, Of, Sh, ., R.K, ., Sharma, ,,...","[0, 2, 2, 1, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, ..."
8392,"[Petitioner, :, Bhagwant, Singh, Vs, ., Respon...","[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, ..."


In [22]:
# concatinating gold test samples + legalBART samples
legalBARTSamples = pd.concat([test_df, legalBARTSamples])

legalBARTSamplesDataset = Dataset.from_pandas(legalBARTSamples)
legalBARTSamples_tokenized = legalBARTSamplesDataset.map(tokenize_all_labels, batched=True)
legalBARTSamples_results = trainer.evaluate(legalBARTSamples_tokenized)
legalBARTSamples_results

Map:   0%|          | 0/9261 [00:00<?, ? examples/s]

{'eval_loss': 5.538338661193848,
 'eval_precision': 0.05927549496603882,
 'eval_recall': 0.013810131732158639,
 'eval_f1': 0.022401186962527395,
 'eval_accuracy': 0.39327784837052837,
 'eval_runtime': 93.0762,
 'eval_samples_per_second': 99.499,
 'eval_steps_per_second': 19.908,
 'epoch': 5.0}

In [23]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.39328


# Evaluating DAGA Samples

In [24]:
wandb.init(name = "daga-test")

In [25]:
# loading DAGA samples
DAGASamples = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/daga_samples_postprocessed.csv')

DAGASamples['tokens'] = DAGASamples['tokens'].map(lambda x: eval(x))
DAGASamples['ner_tags'] = DAGASamples['ner_tags'].map(lambda x: eval(x))

In [26]:
# concatinating gold test samples + DAGA samples
DAGASamples = pd.concat([test_df, DAGASamples])

DAGASamplesDataset = Dataset.from_pandas(DAGASamples)
DAGASamples_tokenized = DAGASamplesDataset.map(tokenize_all_labels, batched=True)
DAGASamples_results = trainer.evaluate(DAGASamples_tokenized)
DAGASamples_results

Map:   0%|          | 0/13332 [00:00<?, ? examples/s]

{'eval_loss': 1.1072564125061035,
 'eval_precision': 0.5517596213212596,
 'eval_recall': 0.4887504747436384,
 'eval_f1': 0.5183472565715804,
 'eval_accuracy': 0.8318846555908833,
 'eval_runtime': 93.8272,
 'eval_samples_per_second': 142.091,
 'eval_steps_per_second': 28.425,
 'epoch': 5.0}

In [27]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.83188


# Evaluating MulDA Samples

In [28]:
wandb.init(name = "mulda-test")

In [29]:
# loading MulDa samples
MulDaSamples_train = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/train_postprocessed.csv')
MulDaSamples_test = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/test_postprocessed.csv')
MulDaSamples_dev = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/dev_postprocessed.csv')

MulDaSamples_train_val = MulDaSamples_train.append(MulDaSamples_dev, ignore_index=True)
MulDaSamples = MulDaSamples_train_val.append(MulDaSamples_test, ignore_index=True)

MulDaSamples['tokens'] = MulDaSamples['tokens'].map(lambda x: eval(x))
MulDaSamples['ner_tags'] = MulDaSamples['ner_tags'].map(lambda x: eval(x))

In [30]:
MulDaSamples

,tokens,ner_tags
0,"[that, extent.]","[0, 0]"
1,"[The, jurisdiction, granted, to, the, Tribunal...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 0, 0, 0, ..."
2,"[In, the, said, case,, Kailasm, ,, J.,, while,...","[0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[In, Bhinka, v., Charan, Singh,, (AIR, 1959, S...","[0, 12, 12, 12, 12, 12, 12, 12, 12, 0, 0, 0, 0..."
4,"[Thereafter,, he, shouted, and, his, aunt, Gau...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, ..."
...,...,...
11111,"[Carbp-196, of, 2016.odt, rrpillai, In, The, H...","[0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, ..."
11112,"[dss, 1, Judgment-wp-10835-18-g.doc, In, The, ...","[0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, ..."
11113,"[In, The, High, Court, Of, Kerala, At, Ernakul...","[0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 4, 4, ..."
11114,"[High, Court, Of, Judicature, At, Allahabad,, ...","[2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."


In [31]:
# concatinating gold test samples + MulDa samples
MulDaSamples = pd.concat([test_df, MulDaSamples])

MulDaSamplesDataset = Dataset.from_pandas(MulDaSamples)
MulDaSamples_tokenized = MulDaSamplesDataset.map(tokenize_all_labels, batched=True)
MulDaSamples_results = trainer.evaluate(MulDaSamples_tokenized)
MulDaSamples_results

Map:   0%|          | 0/11983 [00:00<?, ? examples/s]

{'eval_loss': 0.38587263226509094,
 'eval_precision': 0.5388313311295654,
 'eval_recall': 0.6853344955327958,
 'eval_f1': 0.6033164632666993,
 'eval_accuracy': 0.8184640364594145,
 'eval_runtime': 145.2707,
 'eval_samples_per_second': 82.487,
 'eval_steps_per_second': 16.5,
 'epoch': 5.0}

In [32]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.81846


# Extras: Demonstrating Results on an Indian Legal Example

In [33]:
# importing the trained model from the checkpoint
tokenizer = AutoTokenizer.from_pretrained('./indian_legal_ner.model/')
model = AutoModelForTokenClassification.from_pretrained('./indian_legal_ner.model/', num_labels=len(labels_list))
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [34]:
sentence = '''Further, section 19(3) of the act specifies that unless a different intention appears, rules contained in section 20 to 24 rules for ascertaining the intention of the parties as to the time at which the property and goods is to passed to the buyer.'''
tokenizer = AutoTokenizer.from_pretrained('./indian_legal_ner.model/')
input_tensor = tokenizer(sentence)['input_ids']
attention_tensor = tokenizer(sentence)['attention_mask']

model = AutoModelForTokenClassification.from_pretrained('./indian_legal_ner.model/', num_labels=len(labels_list))

predictions = model.forward(input_ids=torch.tensor(input_tensor).unsqueeze(0), attention_mask=torch.tensor(attention_tensor).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
entities = [labels_list[i] for i in predictions]

tokens = tokenizer.batch_decode(input_tensor)
for tag, entity, token in zip(predictions, entities, tokens):
  print(f"{token:<12}{tag:<12}{entity}")

[CLS]       0            OTHERS
further     0            OTHERS
,           0            OTHERS
section     11           PROVISION
19          11           PROVISION
(           11           PROVISION
3           11           PROVISION
)           11           PROVISION
of          0            OTHERS
the         0            OTHERS
act         0            OTHERS
specifie    0            OTHERS
##s         0            OTHERS
that        0            OTHERS
unless      0            OTHERS
a           0            OTHERS
different   0            OTHERS
intention   0            OTHERS
appears     0            OTHERS
,           0            OTHERS
rules       0            OTHERS
contained   0            OTHERS
in          0            OTHERS
section     11           PROVISION
20          11           PROVISION
to          11           PROVISION
24          11           PROVISION
rules       0            OTHERS
for         0            OTHERS
ascertain   0            OTHERS
##ing       0